In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import scipy.optimize as opt
from scipy.optimize import curve_fit
import os
import re


In [3]:
def powerlaw(t,K,n,Af,tau):
    return Af*(1-np.exp(-K*np.power(t+tau,n)/Af))#/Af))


In [ ]:
file_temp=next(os.walk('.'))[1]
cst=0
f_name=[""]
f_id=[""]
for temp in file_temp:
    match=re.search("data_*",temp)
    if match:
        f_name.append(temp)
        f_id.append(temp.split('_')[1])
        f_id.append(temp.split('_')[2])
        
        cst=cst+1
f_name=np.array(f_name[1:])#,dtype=np.str)
f_id=np.array(f_id[1:],dtype=np.float32)
f_sorted=np.sort(f_id)
index=np.zeros(np.size(f_id)).astype('int32')
f_name_sorted=np.empty(np.size(f_name),dtype='<U14')
for i in range(np.size(f_id)):
    index[i]=np.where(f_id==f_sorted[i])[0][0].astype('int32')#,f_id)
    #f_name_sorted[i]=f_name[index[i]]
n=np.size(f_id)
fit_param=np.zeros([n,4])
i=0
for f in f_name:
    temp=pd.read_csv('./'+f+'/dropinfo',header=0,delimiter='\\s+')#,skiprows=1)
    data=temp[1:].astype('float32') # Drop first row and interpret data as floating point numbers. 
    # === Plot Data === #
    xdata=data['Time'].to_numpy()
    ydata=np.pi*np.power(data['Wetted'].to_numpy(),2)
    p=plt.plot(xdata,ydata,label=f.split('_')[1])
    # === Fit model === #
    #popt, pcov = curve_fit(powerlaw, data['Time'], np.pi*np.power(data['Wetted'],2),[1,0.5,1,8])
    popt, pcov = curve_fit(powerlaw, xdata, ydata,bounds=(10**-3,[10,10,10,2]))
    fit_param[i,:]=popt
    p=plt.plot(xdata,powerlaw(xdata,popt[0],popt[1],popt[2],popt[3]),'--',color=p[0].get_color())
    i=i+1

plt.ylim([0,np.pi*np.power(np.max(data['Wetted']),2)*1.2])
plt.title('Droplet Spreading')
plt.xlabel('time')
plt.ylabel('Area')
plt.legend()
plt.grid()
plt.show()
    

In [ ]:
# K,n,Af,tau
a=plt.plot(np.log10(f_id),fit_param[:,0]/np.max(fit_param[:,0]),'-o',label='K')
a=plt.plot(np.log10(f_id),fit_param[:,1]/np.max(fit_param[:,1]),'-o',label='n')
a=plt.plot(np.log10(f_id),fit_param[:,2]/np.max(fit_param[:,2]),'-o',label='Af')
a=plt.plot(np.log10(f_id),fit_param[:,3]/np.max(fit_param[:,3]),'-o',label='tau')
plt.title('beta')
plt.xlabel('log beta')
plt.ylabel('normalized fit variables')
plt.legend()

plt.grid()
plt.show()

In [5]:
f_name

array(['data_0_0', 'data_1_0', 'data_1_1', 'data_0_1'], dtype='<U8')

In [36]:
# K,n,Af,tau
fit_param

array([[5.30037902e-01, 6.30834679e-01, 3.59550164e+00, 3.42524170e-02],
       [2.03964475e-01, 2.13751348e+00, 7.74362696e+00, 6.95567387e-01],
       [1.49548545e+00, 1.46967570e+00, 8.10311914e+00, 1.00000000e-03],
       [4.49041604e+00, 1.05884132e+00, 8.29380617e+00, 1.00000000e-03],
       [5.97011383e+00, 8.81841051e-01, 8.38774281e+00, 1.07507007e-02]])

In [40]:
f_sorted

array([0.05, 0.1 , 0.2 , 0.4 , 0.6 , 0.8 , 1.  , 5.  ], dtype=float32)

In [18]:
f_name


array(['monitor2_0.3', 'monitor2_0.05', 'monitor2_0.01', 'monitor2_0.1',
       'monitor2_1'], dtype='<U13')